# 0. Import Library

In [1]:
import numpy as np
import pandas as pd
import random

from zipfile import ZipFile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import matplotlib.pyplot as plt

import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

2025-05-18 14:00:51.246450: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747576851.477957      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747576851.546160      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# 1. Business Understanding

# 2. Data Understanding

## 2.1 Load Data dan Cetak Atribut Tabel

In [2]:
dataUser = pd.read_csv("/kaggle/input/book-reccomendation-dataset/Data/Users.csv")
dataBuku = pd.read_csv("/kaggle/input/book-reccomendation-dataset/Data/Books.csv")
dataRating = pd.read_csv("/kaggle/input/book-reccomendation-dataset/Data/Ratings.csv")

dataUser.info()
dataBuku.info()
dataRating.info()

/tmp/ipykernel_19/3745290253.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  dataBuku = pd.read_csv("/kaggle/input/book-reccomendation-dataset/Data/Books.csv")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271358 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)


## 2.2 Identifikasi Karakteristik Data dan Cetak Ringkasan Statistik

In [3]:
dataUser = dataUser.rename(columns={
    'User-ID': 'userID'
})
dataRating = dataRating.rename(columns={
    'User-ID': 'userID'
})

print('Banyak User : ', len(dataUser.userID.unique()))
print('Banyak Buku : ', len(dataBuku.ISBN.unique()))
print('Banyak User yang Memberi Rating : ', len(dataRating.userID.unique()))

Banyak User :  278858
Banyak Buku :  271360
Banyak User yang Memberi Rating :  105283


## 2.3 Identifikasi Ringkasan Statistik Data

### 2.3.1 Ringkasan Statistik Dataset Books

In [4]:
dataBuku.drop(columns=['Image-URL-S','Image-URL-M','Image-URL-L','ISBN']).describe(include='all')

,Book-Title,Book-Author,Year-Of-Publication,Publisher
count,271360,271358,271360,271358
unique,242135,102022,202,16807
top,Selected Poems,Agatha Christie,2002,Harlequin
freq,27,632,13903,7535


### 2.3.2 Ringkasan Statistik Dataset Ratings

In [5]:
dataRating.drop(columns=['userID']).describe(include='all')

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,ISBN,Book-Rating
count,1149780,1.149780e+06
unique,340556,NaN
top,0971880107,NaN
freq,2502,NaN
mean,NaN,2.866950e+00
std,NaN,3.854184e+00
min,NaN,0.000000e+00
25%,NaN,0.000000e+00
50%,NaN,0.000000e+00
75%,NaN,7.000000e+00


### 2.3.3 Ringkasan Statistik Dataset Users

In [6]:
dataUser.drop(columns=['userID']).describe(include='all')

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Location,Age
count,278858,168096.000000
unique,57339,NaN
top,"london, england, united kingdom",NaN
freq,2506,NaN
mean,NaN,34.751434
std,NaN,14.428097
min,NaN,0.000000
25%,NaN,24.000000
50%,NaN,32.000000
75%,NaN,44.000000


## 2.4 Pengecekan Missing Value

In [7]:
dataUser.isnull().sum()

userID           0
Location         0
Age         110762
dtype: int64

In [8]:
dataBuku.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [9]:

dataRating.isnull().sum()

userID         0
ISBN           0
Book-Rating    0
dtype: int64

# 3. Data Preparation

## 3.1 Join Tabel

In [10]:
# Gabungkan Data Rating dengan Data User untuk memperoleh dataset rating beserta atribut usernya
dataRatingUser = pd.merge(dataRating, dataUser, on='userID', how='left')
# Gabungkan dataset sebelumnya dengan dataset buku untuk memperoleh keterangan buku yang diberi rating
dataset = pd.merge(dataRatingUser, dataBuku, on='ISBN', how='left')

In [11]:
dataset.info()
dataset.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 12 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   userID               1149780 non-null  int64  
 1   ISBN                 1149780 non-null  object 
 2   Book-Rating          1149780 non-null  int64  
 3   Location             1149780 non-null  object 
 4   Age                  840288 non-null   float64
 5   Book-Title           1031136 non-null  object 
 6   Book-Author          1031134 non-null  object 
 7   Year-Of-Publication  1031136 non-null  object 
 8   Publisher            1031134 non-null  object 
 9   Image-URL-S          1031136 non-null  object 
 10  Image-URL-M          1031136 non-null  object 
 11  Image-URL-L          1031132 non-null  object 
dtypes: float64(1), int64(2), object(9)
memory usage: 105.3+ MB


userID                      0
ISBN                        0
Book-Rating                 0
Location                    0
Age                    309492
Book-Title             118644
Book-Author            118646
Year-Of-Publication    118644
Publisher              118646
Image-URL-S            118644
Image-URL-M            118644
Image-URL-L            118648
dtype: int64

## 3.2 Eliminasi Observasi dengan Missing Value

In [12]:
dataFinal = dataset.dropna()

### 3.3.1 Preprocessing untuk Content Based Filtering

In [13]:
# Membentuk Variabel Baru yang menangkap atribut dari Buku
dataFinal['AtributBuku'] = dataFinal['Book-Title'] + ' ' + dataFinal['Book-Author'] + ' ' + dataFinal['Year-Of-Publication'].astype(str)

# Preprocessing: Convert to lowercase, remove punctuation
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    return text

dataFinal['AtributBuku'] = dataFinal['AtributBuku'].apply(preprocess_text)

# Undersampling untuk Reduksi Dataset
dataBaru = dataFinal.dropna(subset=['AtributBuku']).sample(n=90000, random_state=42).reset_index(drop=True)

/tmp/ipykernel_19/3961848135.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFinal['AtributBuku'] = dataFinal['Book-Title'] + ' ' + dataFinal['Book-Author'] + ' ' + dataFinal['Year-Of-Publication'].astype(str)
/tmp/ipykernel_19/3961848135.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFinal['AtributBuku'] = dataFinal['AtributBuku'].apply(preprocess_text)


### 3.3.2 Preprocessing untuk Colaborative Filtering

In [14]:
# Mengubah userID menjadi list unik
user_ids = dataFinal['userID'].unique().tolist()
user_to_user_encoded = {x: i for i, x in enumerate(user_ids)}
user_encoded_to_user = {i: x for i, x in enumerate(user_ids)}

# Mengubah ISBN menjadi list unik
buku_ids = dataFinal['ISBN'].unique().tolist()
buku_to_ids_encoded = {x: i for i, x in enumerate(buku_ids)}
ids_to_buku_encoded = {i: x for i, x in enumerate(buku_ids)}

# Melakukan encoding ke dataframe
dataFinal['user_encoded'] = dataFinal['userID'].map(user_to_user_encoded)
dataFinal['ISBN_encoded'] = dataFinal['ISBN'].map(buku_to_ids_encoded)

# Cek apakah ada yang gagal di-encode
if dataFinal[['user_encoded', 'ISBN_encoded']].isnull().values.any():
    print("🚨 Ada userID atau ISBN yang tidak berhasil di-encode!")
    print("Jumlah user_encoded NaN:", dataFinal['user_encoded'].isnull().sum())
    print("Jumlah ISBN_encoded NaN:", dataFinal['ISBN_encoded'].isnull().sum())
    # Bisa drop baris NaN atau isi dengan nilai default
    dataFinal = dataFinal.dropna(subset=['user_encoded', 'ISBN_encoded'])

# Convert ke integer untuk keperluan input model embedding
dataFinal['user_encoded'] = dataFinal['user_encoded'].astype(np.int32)
dataFinal['ISBN_encoded'] = dataFinal['ISBN_encoded'].astype(np.int32)

# Konversi rating ke float32
dataFinal['Book-Rating'] = dataFinal['Book-Rating'].astype(np.float32)

# Info statistik
num_users = len(user_to_user_encoded)
num_items = len(ids_to_buku_encoded)
min_rating = dataFinal['Book-Rating'].min()
max_rating = dataFinal['Book-Rating'].max()

print(f'Jumlah User: {num_users}, Jumlah Buku: {num_items}, Min Rating: {min_rating}, Max Rating: {max_rating}')

/tmp/ipykernel_19/3581592387.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFinal['user_encoded'] = dataFinal['userID'].map(user_to_user_encoded)
/tmp/ipykernel_19/3581592387.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFinal['ISBN_encoded'] = dataFinal['ISBN'].map(buku_to_ids_encoded)


Jumlah User: 53887, Jumlah Buku: 228995, Min Rating: 0.0, Max Rating: 10.0


/tmp/ipykernel_19/3581592387.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFinal['user_encoded'] = dataFinal['user_encoded'].astype(np.int32)
/tmp/ipykernel_19/3581592387.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFinal['ISBN_encoded'] = dataFinal['ISBN_encoded'].astype(np.int32)
/tmp/ipykernel_19/3581592387.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

In [15]:
# Mengacak dataset
df = dataFinal.sample(frac=1, random_state=42)
df

# Membuat variabel x untuk mencocokkan data user dan resto menjadi satu value
x = df[['user_encoded', 'ISBN_encoded']].values
 
# Membuat variabel y untuk membuat rating dari hasil 
y = df['Book-Rating'].apply(lambda x: (x - min_rating) / (max_rating - min_rating)).values
 
# Membagi menjadi 80% data train dan 20% data validasi
train_indices = int(0.8 * df.shape[0])
x_train, x_val, y_train, y_val = (
    x[:train_indices],
    x[train_indices:],
    y[:train_indices],
    y[train_indices:]
)

# 4. Modelling

## 4.1 Content Based Filtering

### 4.1.1 Membentuk Matriks Term Frequency - Inverse Document Frequency

In [16]:
# Buat dataframe baru berisi data unik dari AtributBuku
dataBaru = dataBaru.drop_duplicates(subset='AtributBuku').reset_index(drop=False)

tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(dataBaru['AtributBuku'])

### 4.1.2 Membentuk Matriks Cosine Similarity

In [17]:
cosine_sim = cosine_similarity(tfidf_matrix, dense_output=False)


### 4.1.3 Pendefinisian Fungsi Rekomendasi Produk

In [18]:
def recommend(book_title, cosine_sim=cosine_sim, jmlbuku = 5):
    print(f"Menampilkan Rekomendasi Buku yang sesuai dengan '{book_title}' ...")
    
    matches = dataBaru[dataBaru['Book-Title'] == book_title]
    if matches.empty:
        return f"Tidak ditemukan judul: {book_title}"
    
    # matches.index sudah 0..n-1, cocok untuk cosine_sim
    idx = matches.index[0]

    sim_scores_row = cosine_sim[idx].toarray().flatten()

    sim_scores = list(enumerate(sim_scores_row))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:jmlbuku+1]  # skip buku itu sendiri

    book_indices = [i[0] for i in sim_scores]
    scores = [i[1] for i in sim_scores]

    result = dataBaru.iloc[book_indices][['Book-Title', 'Book-Author', 'Publisher', 'Year-Of-Publication']].copy()
    result['similarity_score'] = scores

    return result.reset_index(drop=True)


In [19]:
recommend(dataBaru['Book-Title'].dropna().sample(1).iloc[0])


Menampilkan Rekomendasi Buku yang sesuai dengan 'The Forsyte Saga (Penguin Modern Classics)' ...


,Book-Title,Book-Author,Publisher,Year-Of-Publication,similarity_score
0,Forsyte Saga,John Galsworthy,Touchstone,1996,0.815395
1,Man of Property: The Forsyte Saga,J. Galsworthy,Lb May &amp; Assoc Inc,2001,0.684410
2,The Forsyte Saga : The Man of Property and In ...,John Galsworthy,Touchstone,2002,0.607280
3,In Chancery,John Galsworthy,Lb May &amp; Assoc Inc,1999,0.391592
4,Of Mice and Men (Penguin Modern Classics),John Steinbeck,Penguin Books Ltd,2004,0.348258


## 4.2 Collaborative Filtering

### 4.2.1 Pendefinisian Model RecommenderNet (Neural Colaborative Filtering)

In [20]:
import tensorflow as tf
from tensorflow.keras import layers, regularizers, Model

class RecommenderNet(Model):

    def __init__(self, num_users, num_buku, embedding_size, dropout_rate=0.3, **kwargs):
        super(RecommenderNet, self).__init__(**kwargs)
        self.num_users = num_users
        self.num_buku = num_buku
        self.embedding_size = embedding_size
        self.dropout_rate = dropout_rate

        # User Embedding Layer
        self.user_embedding = layers.Embedding(
            input_dim=num_users,
            output_dim=embedding_size,
            embeddings_initializer='he_normal',
            embeddings_regularizer=regularizers.l2(1e-6)
        )
        self.user_bias = layers.Embedding(input_dim=num_users, output_dim=1)

        # Item Embedding Layer
        self.buku_embedding = layers.Embedding(
            input_dim=num_buku,
            output_dim=embedding_size,
            embeddings_initializer='he_normal',
            embeddings_regularizer=regularizers.l2(1e-6)
        )
        self.buku_bias = layers.Embedding(input_dim=num_buku, output_dim=1)

        # Dropout Layer
        self.dropout = layers.Dropout(dropout_rate)

        # Fully Connected Layer
        self.fc1 = layers.Dense(embedding_size, activation='relu')
        self.batch_norm = layers.BatchNormalization()
        self.fc2 = layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        # Extract user and item embeddings
        user_vector = self.user_embedding(inputs[:, 0])
        user_bias = self.user_bias(inputs[:, 0])
        buku_vector = self.buku_embedding(inputs[:, 1])
        buku_bias = self.buku_bias(inputs[:, 1])

        # Normalize embeddings
        user_vector = tf.nn.l2_normalize(user_vector, axis=1)
        buku_vector = tf.nn.l2_normalize(buku_vector, axis=1)

        # Compute dot product
        dot_product = tf.reduce_sum(user_vector * buku_vector, axis=1, keepdims=True)

        # Add biases
        x = dot_product + user_bias + buku_bias

        # Apply dropout and dense layers
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.batch_norm(x)
        x = self.fc2(x)

        return x

In [ ]:
model = RecommenderNet(num_users, num_items, 512) # inisialisasi model
 
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00005),
    loss='mse',
    metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')]
)

I0000 00:00:1747576881.408711      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


### 4.2.2 Pelatihan Model Rekomendasi

In [22]:
# Memulai training
 
history = model.fit(
    x = x_train,
    y = y_train,
    batch_size = 2048,
    epochs = 100,
    validation_data = (x_val, y_val)
)

Epoch 1/100


/usr/local/lib/python3.11/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 117245440 elements. This may consume a large amount of memory.
  warnings.warn(
I0000 00:00:1747576886.264252      60 service.cc:148] XLA service 0x7b79e400d510 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747576886.265139      60 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1747576886.560810      60 cuda_dnn.cc:529] Loaded cuDNN version 90300


  8/295 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - loss: 0.1992 - rmse: 0.4440

I0000 00:00:1747576887.627367      60 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


295/295 ━━━━━━━━━━━━━━━━━━━━ 13s 24ms/step - loss: 0.1763 - rmse: 0.4173 - val_loss: 0.1509 - val_rmse: 0.3863
Epoch 2/100
295/295 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - loss: 0.1259 - rmse: 0.3522 - val_loss: 0.1376 - val_rmse: 0.3685
Epoch 3/100
295/295 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - loss: 0.0292 - rmse: 0.1654 - val_loss: 0.1331 - val_rmse: 0.3623
Epoch 4/100
295/295 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - loss: 0.0132 - rmse: 0.1068 - val_loss: 0.1323 - val_rmse: 0.3614
Epoch 5/100
295/295 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - loss: 0.0133 - rmse: 0.1076 - val_loss: 0.1318 - val_rmse: 0.3607
Epoch 6/100
295/295 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - loss: 0.0116 - rmse: 0.0996 - val_loss: 0.1315 - val_rmse: 0.3603
Epoch 7/100
295/295 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - loss: 0.0107 - rmse: 0.0947 - val_loss: 0.1313 - val_rmse: 0.3601
Epoch 8/100
295/295 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - loss: 0.0096 - rmse: 0.0892 - val_loss: 0.1312 - val_rmse: 0.3599
Epoch 9/100
295/295 ━━━━━━━━━━━━━━━

### 4.2.3 Visualisasi Hasil Pelatihan

In [23]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model_metrics')
plt.ylabel('Akurasi')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

KeyError: 'accuracy'

### 4.2.4 Merekomendasikan Buku

In [ ]:
def recommend_books_for_user(model, user_id_asli, data, buku_df, user_to_user_encoded, buku_to_ids_encoded, ids_to_buku_encoded, top_n=10):
    # Cek apakah user_id valid
    if user_id_asli not in user_to_user_encoded:
        print("User ID tidak ditemukan.")
        return []

    user_encoded = user_to_user_encoded[user_id_asli]
    buku_user_sudah_rating = data[data['userID'] == user_id_asli]['ISBN'].tolist()
    buku_user_sudah_encoded = [buku_to_ids_encoded[buku] for buku in buku_user_sudah_rating]

    # Semua buku yang belum diberi rating oleh user
    buku_tidak_dirating = list(set(buku_to_ids_encoded.values()) - set(buku_user_sudah_encoded))
    user_encoded_array = np.full(len(buku_tidak_dirating), user_encoded)
    
    # Membuat data pasangan user-buku
    input_pairs = np.array(list(zip(user_encoded_array, buku_tidak_dirating)))

    # Prediksi skor
    ratings = model.predict(input_pairs).flatten()
    
    # Ambil top-N
    top_indices = ratings.argsort()[-top_n:][::-1]
    top_buku_encoded = [buku_tidak_dirating[i] for i in top_indices]
    top_buku_isbn = [ids_to_buku_encoded[i] for i in top_buku_encoded]

    # Gabungkan dengan info buku (jika ada dataframe buku_df)
    rekomendasi = buku_df[buku_df['ISBN'].isin(top_buku_isbn)].copy() if buku_df is not None else pd.DataFrame({'ISBN': top_buku_isbn})
    rekomendasi['Predicted_Score'] = ratings[top_indices]
    
    return rekomendasi.sort_values(by='Predicted_Score', ascending=False)

# Menampilkan 5 userID unik pertama dari data
sample_user_ids = dataFinal['userID'].unique()[:5]
user_id_sample = sample_user_ids[0]  # atau pilih indeks lain

# Contoh penggunaan:
rekomendasi_df = recommend_books_for_user(model, user_id_asli=user_id_sample, data=dataFinal, buku_df=dataBuku,
    user_to_user_encoded=user_to_user_encoded,
    buku_to_ids_encoded=buku_to_ids_encoded,
    ids_to_buku_encoded=ids_to_buku_encoded,
    top_n=5)

# Tampilkan hasil
display(rekomendasi_df)


# 5. Evaluasi

## 5.1 Evaluasi Content Based Filtering : Cosine Similarity

In [ ]:
def evaluate_cbf_recommender(dataBaru, recommend_fn, users_to_evaluate, k=5):
    results = []
    user_books = dataBaru.groupby("userID")["Book-Title"].apply(set).to_dict()

    for user in users_to_evaluate:
        books_read = list(user_books.get(user, []))
        if not books_read:
            continue
        seed_book = pd.Series(books_read).dropna().sample(1).iloc[0]
        try:
            rec_df = recommend_fn(seed_book, cosine_sim, len(books_read))
        except:
            continue
        rec_books = rec_df["Book-Title"].tolist()[:k]
        actual_books = set(books_read)
        relevance_vector = [1 if book in actual_books else 0 for book in rec_books]
        true_positives = sum(relevance_vector)

        precision = true_positives / k if k > 0 else 0
        recall = true_positives / len(actual_books) if actual_books else 0
        f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        hit_rate = 1 if true_positives > 0 else 0

        results.append({
            "userID": user,
            "seed_book": seed_book,
            "recommended": rec_books,
            "actual": list(actual_books),
            "relevance_vector": relevance_vector,
            "precision@k": precision,
            "recall@k": recall,
            "f1_score@k": f1_score,
            "hit_rate@k": hit_rate
        })

    results_df = pd.DataFrame(results)
    avg_precision = results_df["precision@k"].mean()
    avg_recall = results_df["recall@k"].mean()
    avg_f1 = results_df["f1_score@k"].mean()
    avg_hit_rate = results_df["hit_rate@k"].mean()

    return results_df, avg_precision, avg_recall, avg_f1, avg_hit_rate

import random

# Hitung jumlah buku per user
user_book_counts = dataBaru.groupby("userID")["Book-Title"].nunique()

# Filter user dengan minimal 5 buku
eligible_users = user_book_counts[user_book_counts >= 5].index.tolist()

# Ambil sample user dari yang eligible
sample_users = random.sample(eligible_users, 10)

# Jalankan evaluasi
results_df, avg_prec, avg_rec, avg_f1, avg_hr = evaluate_cbf_recommender(dataBaru, recommend, sample_users, k=5)

print(f"Avg Precision@5: {avg_prec:.4f}")
print(f"Avg Recall@5: {avg_rec:.4f}")
print(f"Avg F1-Score@5: {avg_f1:.4f}")
print(f"Avg Hit Rate@5: {avg_hr:.4f}")

results_df.head()


## 5.2 Evaluasi Colaborative Filtering

In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np

def evaluate_cf_model(
    model, 
    x_val, 
    y_val, 
    dataFinal, 
    buku_df,
    user_to_user_encoded, 
    buku_to_ids_encoded, 
    ids_to_buku_encoded, 
    k=5
):
    # Buat DataFrame val dari x_val dan y_val
    val_df = pd.DataFrame({
        "user_encoded": x_val[:, 0],
        "buku_encoded": x_val[:, 1],
        "interaction": y_val
    })

    # Filter interaksi positif
    val_pos_df = val_df[val_df["interaction"] > 0]

    # Mapping user_encoded -> set buku_encoded
    user_items_val = val_pos_df.groupby("user_encoded")["buku_encoded"].apply(set).to_dict()

    # Ambil 10 user secara acak
    users_to_eval = list(user_items_val.keys())
    if len(users_to_eval) > 10:
        users_to_eval = random.sample(users_to_eval, 10)

    results = []

    for user_encoded in users_to_eval:
        user_id_asli = next((k_ for k_, v_ in user_to_user_encoded.items() if v_ == user_encoded), None)
        if user_id_asli is None:
            continue

        actual_items = user_items_val[user_encoded]

        rekomendasi_df = recommend_books_for_user(
            model=model,
            user_id_asli=user_id_asli,
            data=dataFinal,
            buku_df=buku_df,
            user_to_user_encoded=user_to_user_encoded,
            buku_to_ids_encoded=buku_to_ids_encoded,
            ids_to_buku_encoded=ids_to_buku_encoded,
            top_n=k
        )

        rec_books_encoded = [
            buku_to_ids_encoded[isbn] for isbn in rekomendasi_df['ISBN'] if isbn in buku_to_ids_encoded
        ] if not rekomendasi_df.empty else []

        relevance_vector = [1 if book in actual_items else 0 for book in rec_books_encoded]
        true_positives = sum(relevance_vector)

        precision = true_positives / k if k > 0 else 0
        recall = true_positives / len(actual_items) if actual_items else 0
        f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        hit_rate = 1 if true_positives > 0 else 0

        results.append({
            "user_encoded": user_encoded,
            "user_id_asli": user_id_asli,
            "recommended_encoded": rec_books_encoded,
            "actual_encoded": list(actual_items),
            "relevance_vector": relevance_vector,
            "precision@k": precision,
            "recall@k": recall,
            "f1_score@k": f1_score,
            "hit_rate@k": hit_rate
        })

    results_df = pd.DataFrame(results)
    avg_precision = results_df["precision@k"].mean()
    avg_recall = results_df["recall@k"].mean()
    avg_f1 = results_df["f1_score@k"].mean()
    avg_hit_rate = results_df["hit_rate@k"].mean()

    # Hitung RMSE pada seluruh validation set
    y_pred = model.predict(x_val).flatten()
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))

    return results_df, avg_precision, avg_recall, avg_f1, avg_hit_rate, rmse

In [ ]:
results_df, avg_prec, avg_rec, avg_f1, avg_hr, rmse = evaluate_cf_model(
    model, x_val, y_val, dataFinal, dataBuku,
    user_to_user_encoded, buku_to_ids_encoded, ids_to_buku_encoded,
    k=5
)

print(f"Avg Precision@5: {avg_prec:.4f}")
print(f"Avg Recall@5: {avg_rec:.4f}")
print(f"Avg F1-Score@5: {avg_f1:.4f}")
print(f"Avg Hit Rate@5: {avg_hr:.4f}")
print(f"RMSE: {rmse:.4f}")
